In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F

from netam import framework, models
from netam.framework import calculate_loss
from epam.torch_common import pick_device

from epam.torch_common import PositionalEncoding

/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
shmoof_data_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
train_df, val_df = framework.load_shmoof_dataframes(shmoof_data_path, val_nickname="59") #, sample_count=5000)


In [3]:
kmer_length = 3
max_length = 410

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

Using Metal Performance Shaders
we have 44330 training examples and 4686 validation examples


In [4]:
class CNNPPModel(nn.Module):
    def __init__(self, dataset, embedding_dim, num_filters, kernel_size, dropout_rate):
        super(CNNPPModel, self).__init__()
        self.kmer_count = len(dataset.kmer_to_index)
        self.kmer_embedding = nn.Embedding(self.kmer_count, embedding_dim)
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout=dropout_rate) 
        self.conv = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=kernel_size, padding='same')
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(in_features=num_filters, out_features=1)


    def forward(self, encoded_parents, masks):
        kmer_embeds = self.kmer_embedding(encoded_parents)
        kmer_embeds = self.pos_encoder(kmer_embeds)
        kmer_embeds = kmer_embeds.permute(0, 2, 1)  # Transpose for Conv1D
        conv_out = F.relu(self.conv(kmer_embeds))
        conv_out = self.dropout(conv_out)
        conv_out = conv_out.permute(0, 2, 1)  # Transpose back for Linear layer
        log_rates = self.linear(conv_out).squeeze(-1)
        rates = torch.exp(log_rates * masks)
        return rates


model = CNNPPModel(train_dataset, embedding_dim=10, num_filters=9, kernel_size=11, dropout_rate=0.1)
model.to(device)


CNNPPModel(
  (kmer_embedding): Embedding(65, 10)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (conv): Conv1d(10, 9, kernel_size=(11,), stride=(1,), padding=same)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear): Linear(in_features=9, out_features=1, bias=True)
)

In [5]:
burrito = framework.Burrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, min_learning_rate=1e-4, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=100)
losses

starting training...


Epoch:  47%|████▋     | 47/100 [07:59<09:00, 10.20s/it, loss_diff=8.853e-08, lr=3.2e-5]  


,train_loss,val_loss
0,0.067934,0.063368
1,0.066426,0.061113
2,0.065325,0.060955
3,0.064786,0.060025
4,0.063781,0.059193
5,0.062944,0.058698
6,0.062498,0.058554
7,0.062246,0.058803
8,0.062053,0.058573
9,0.061686,0.058348


For comparison, here's a simple CNN.

In [7]:
model = models.CNNModel(train_dataset, embedding_dim=10, num_filters=9, kernel_size=11, dropout_rate=0.1)
model.to(device)

burrito = framework.Burrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, min_learning_rate=1e-4, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=100)
losses

starting training...


Epoch:  36%|███▌      | 36/100 [05:57<10:35,  9.94s/it, loss_diff=8.273e-08, lr=3.2e-5]  


,train_loss,val_loss
0,0.066718,0.062412
1,0.063459,0.058582
2,0.061533,0.058262
3,0.061214,0.058330
4,0.061124,0.058286
5,0.061086,0.058141
6,0.061040,0.058301
7,0.061056,0.058131
8,0.061024,0.058290
9,0.061003,0.058262
